In [ ]:
import numpy as np
import torch
from scipy.integrate import simps
import scipy.stats as stats

In [ ]:
def likelihood(overlap_predicted):
    """
    Calculates the likelihood of the observed data given the predicted overlap.
    """
    return stats.norm.pdf(1.0, loc=overlap_predicted, scale=0.1)


In [ ]:
def pop_i (x_grid,psi_ref,psi_sim):
    
    """
    Calculates the overlap of a wavefunction with another.
    """
    
    overlap= abs(x_grid[0]-x_grid[1])*np.sum(np.conj(psi_sim) * psi_ref)
    fitness=np.abs(overlap**2)
    
    return fitness
    

In [ ]:

# Define the prior sampler function
def prior(param_1_prior,param_2_prior):
    """
    Samples parameter values from the prior distribution.
    """
    param_1 = param_1_prior.rvs()
    param_2 = param_2_prior.rvs()
    return [param_1, param_2]

In [ ]:
# Define the acceptance function
def acceptance(current_params, prop_params,x_grid,y_grid,param_1_prior,param_2_prior,pot_params):
    """
    Calculates the acceptance probability for the proposed parameter values.
    """
    
    sig=0.8
    t0=2.0
    t1=3
    k_curr= current_params[0]
    k1_curr= current_params[1]
    k_prop=prop_params[0]
    k1_prop=prop_params[1]
    
    curr_f_params= {'sig':sig,'t0':t0,'t1':t1,'k':k_curr,'k1':k1_curr}
    prop_f_params= {'sig':sig,'t0':t0,'t1':t1,'k':k_prop,'k1':k1_prop}
    
    curr_score=complete_simulation(x_grid,y_grid,pot_params,curr_f_params)
    current_likelihood = likelihood(curr_score)

    prop_score=complete_simulation(x_grid,y_grid,pot_params,prop_f_params)
    proposed_likelihood = likelihood(prop_score)
    
    prior_current = np.prod([param_1_prior.pdf(current_params[0]), param_2_prior.pdf(current_params[1])])
    prior_proposed = np.prod([param_1_prior.pdf(prop_params[0]), param_2_prior.pdf(prop_params[1])])
    
    #prior_current = prior(range_1,range_2)
    #prior_proposed = prior(range_1,range_2)
    
    return min(1, (proposed_likelihood * prior_proposed) / (current_likelihood * prior_current)),prop_score,curr_score